In [1]:
import sqlite3
from sqlite3 import Error

In [8]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def get_cursor(connection):
    try:
        cursor = connection.cursor()
        print("cursor creation successful")
    except Error as e:
        print(f"the error {e} occurred")
    
    return cursor

def create_table(script_path):
    try:
        sql_file = open(script_path)
        sql_as_string = sql_file.read()
        cursor.executescript(sql_as_string)
        print("table successfully created")
    except Error as e:
        print(f"the error {e} occurred")

In [9]:
connection = create_connection("C:\\10\week11\sample.db")

Connection to SQLite DB successful


In [10]:
cursor = get_cursor(connection)

cursor creation successful


IntegrityError: UNIQUE constraint failed: airports.iata